<a href="https://colab.research.google.com/github/joshheyer/CONAFOR/blob/main/2-Composites_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# Developed for Google LLC by RedCastle Resources Inc
# https://www.redcastleresources.com/

# <img width=50px  src = 'https://apps.fs.usda.gov/lcms-viewer/images/lcms-icon.png'>  Laboratorio 2: Producir Landsat y Sentinel-2 Compuestos Anuales

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/CONAFOR/blob/main/2-Composites_espanol.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/CONAFOR/blob/main/2-Composites_espanol.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>
<br/><br/><br/>

## 2.0: Descripción General e Introducción


Este cuaderno presenta los métodos de composición utilizados por el Sistema de Monitoreo de Cambios del Paisaje (LCMS) del Servicio Forestal del USDA. Luego produce composites para Tabasco México. Los compuestos se utilizarán para generar datos de series de tiempo que los algoritmos de modelado LCMS utilizarán para predecir el uso del suelo, la cobertura del suelo y los cambios en la cobertura vegetal en los módulos posteriores.

**Aprender más sobre [LCMS](https://apps.fs.usda.gov/lcms-viewer/home.html).**


Todos los compuestos se crean y visualizan utilizando el paquete Python `geeViz`. `geeViz` proporciona herramientas de línea de comandos para procesar, analizar y visualizar objetos de Google Earth Engine (GEE). Para la visualización de resultados de mapas, la principal ventaja de "geeViz" es que no depende de IPython. Esto es esencial, ya que todo el procesamiento LCMS operativo generalmente se realiza fuera de las computadoras portátiles, pero debemos poder depurar los resultados y verlos en el mapa.

**Aprender más sobre [geeViz](https://github.com/gee-community/geeViz/tree/master).**


### 2.0.1: Objetivos

#### Este tutorial utiliza los siguientes servicios de Google Cloud:

- `Google Earth Engine`

#### Los pasos realizados incluyen:

- Instalar los paquetes necesarios
- Pruebe varios parámetros para crear compuestos.
    - Ejecutar con parámetros predeterminados
    - Incluir Landsat 7 en los compuestos
    - Establecer la configuración de enmascaramiento de la nube
    - Establecer la configuración de enmascaramiento de sombras de nubes
- Crear y exportar series temporales compuestas a GEE assets

#### Objetivos de aprendizaje:
- Los usuarios crearán compuestos sin nubes y sombras de nubes utilizando Fmask y cloudScore para Landsat, s2Cloudless para Sentinel-2 y máscara de valores atípicos oscuros temporales (TDOM).  



### 2.0.2: Antes de que empiecen

#### Si está trabajando en Workbench: establezca su URL actual en `workbench_url`
Esto le da al Visor de mapas una URL en la que alojar el visor que generaremos.
Esto le da al Visor de mapas una URL en la que alojar el visor que generaremos.
* Esto estará en su URL/barra de búsqueda en la parte superior de la ventana del navegador en la que se encuentra actualmente.
* Se verá algo así como `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (Vea la imagen a continuación).

![workspace url](https://github.com/redcastle-resources/lcms-training/blob/main/img/workspace-url.png?raw=1)

#### Establezca una carpeta para usar en todas las exportaciones en `export_path_root`
* Esta carpeta debe ser una carpeta de activos en un proyecto GEE existente.
* De forma predeterminada, esta carpeta es la misma que la carpeta predefinida (donde ya se han creado los resultados).
* Si desea crear sus propios resultados, especifique una ruta diferente para `export_path_root`, pero deja el`pre_baked_path_root` como esta. De esta manera, las salidas precocidas se pueden mostrar al final, en lugar de esperar a que terminen todas las exportaciones.
* será algo así como `projects/projectID/assets/someFolder`. Utilice el ID de proyecto asociado con su cuenta.
* No es necesario que esta carpeta ya exista. Si no existe, se creará.

In [1]:
workbench_url = 'https://1307eb830a12e633-dot-us-central1.notebooks.googleusercontent.com/'
pre_baked_path_root  = 'projects/ee-jheyer2325/assets'
export_path_root = pre_baked_path_root
print('Hecho')

Done


#### Instalación
Instale los paquetes de Python necesarios. Descomente la primera línea para actualizar geeViz si es necesario.

Tenga en cuenta qué paquetes de Python y componentes de geeViz está importando aquí.

In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

import pandas as pd
import inspect,os

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Hecho')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.2/583.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.5 MB/s eta 0:00:00
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13786 sha256=9f677a77bae1b43ffc0b085c51a560199b58c5354edc13595bd16f321649c7ca
  Stored in directory: /root/.cache/pip/wheels/e5/41/13/ebdef29165b9309ec4e235dbff19eca8b6759125b0924ad430
Successfully built simpledbf
Initializing GEE
No credentials found
Will attempt ee.Authenticate
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bPMfXmRXlJCwyolxG3vA6hTMrIN613t1xGikNgYDjAE&tc=e6ySRrVqcrqcC4j

#### Configura tu entorno de trabajo

Cree una carpeta en su ruta de exportación donde exportará los compuestos. Además, cree una colección de imágenes en blanco donde residirán sus compuestos.

Actualmente, cuando se ejecuta en Colab o Workbench, geeView utiliza un proyecto diferente para autenticarse, por lo que es posible que deba hacer público su recurso para verlo desde Colab.

**Advertencia!!**

* **Se proporciona acceso de solo lectura a todos los usuarios de GEE autenticados para los resultados ya terminados**
* **Si está utilizando la ubicación de salida precocida (`export_path_root = pre_baked_path_root`), Verá errores para cualquier operación que intente escribir, eliminar o cambiar los permisos de acceso a cualquier recurso en la ubicación de salida predefinida.**
* **Esto es lo esperado y no le impedirá ejecutar correctamente este cuaderno. Ignore estos mensajes de error si aparecen.**


In [3]:
# Crear carpeta
export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
print(export_composite_collection)

# Crear una colección
aml.create_asset(export_composite_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Hazlo público
# Esto fallará si está utilizando la ubicación de salida pre_baked_path_root
aml.updateACL(export_composite_collection,writers = [],all_users_can_read = True,readers = [])

print('Hecho')

projects/ee-jheyer2325/assets/lcms-training_module-2_composites
Asset projects/ee-jheyer2325/assets/lcms-training_module-2_composites already exists
Updating permissions for:  projects/ee-jheyer2325/assets/lcms-training_module-2_composites
Done


## 2.1: Explora las funciones de geeViz

A lo largo de los ejemplos de este módulo, se familiarizará más con los parámetros utilizados para crear compuestos de imágenes. La función principal que utilizará es de la biblioteca geeViz y se llama `getLandsatAndSentinel2HybridWrapper`.

Para obtener más documentación sobre los parámetros de la función, consulte la [getCombinedLandsatSentinel2Wrapper](https://github.com/gee-community/geeViz/blob/master/examples/getCombinedLandsatSentinel2Wrapper.py) ejemplo en la documentación de geeViz.

Puedes ver el [source code](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L3280) para esta función en geeViz GitHub.

Mientras que `getLandsatAndSentinel2HybridWrapper` satisfará las necesidades de la mayoría de los flujos de trabajo de composición, a veces es necesario tener más control sobre cómo se crean los compuestos. Para estos casos, debe utilizar las funciones subyacentes que ayudan con el preprocesamiento de datos Landsat o Sentinel-2.:
* [getProcessedLandsatScenes](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L2678)
* [getProcessedSentinel2Scenes](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L2799)

Puede inspeccionar el código de estas funciones siguiendo los enlaces anteriores a la función o utilizando el `print(inspect.getsource(LIBRARY_NAME.FUNCTION_NAME))` dominio. Ejecute el bloque de código a continuación para inspeccionar la función de la biblioteca geeViz que utilizará.

In [14]:
print(inspect.getsource(getImagesLib.getLandsatAndSentinel2HybridWrapper))

def getLandsatAndSentinel2HybridWrapper(\
  studyArea,
  startYear,
  endYear,
  startJulian,
  endJulian,
  timebuffer =  0,
  weights =  [1],
  compositingMethod = 'medoid',
  toaOrSR = 'TOA',
  includeSLCOffL7 = False,
  defringeL5 = False,
  applyQABand = False,
  applyCloudProbability = True,
  applyShadowShift = False,
  applyCloudScoreLandsat = False,
  applyCloudScoreSentinel2 = False,
  applyTDOMLandsat = True,
  applyTDOMSentinel2 = True,
  applyFmaskCloudMask = True,
  applyFmaskCloudShadowMask = True,
  applyFmaskSnowMask = False,
  cloudHeights = ee.List.sequence(500,10000,500),
  cloudScoreThresh = 20,
  performCloudScoreOffset = True,
  cloudScorePctl = 10,
  zScoreThresh = -1,
  shadowSumThresh = 0.35,
  contractPixels = 1.5,
  dilatePixels = 3.5,
  shadowSumBands = ['nir','swir1'],
  landsatResampleMethod = 'near',
  sentinel2ResampleMethod = 'aggregate',
  convertToDailyMosaics = True,
  runChastainHarmonization = True,
  correctIllumination = False,
  correctScale = 

## 2.2: Creer Landsat + Sentinel-2 Compuestos

### 2.2.1: Crear compuestos usando parámetros básicos

#### Configurar mapa
Lo primero que hará es configurar el visor de mapas donde inspeccionará los resultados que genere. No verás el mapa hasta más tarde.

In [5]:
# Configurar proxy url
Map.proxy_url = workbench_url

#Primero borre el mapa en caso de que se haya poblado con capas/comandos anteriormente
Map.clearMap()

# restablecer el puerto si es necesario
Map.port = 1232

print('Hecho')

Done


#### Definir parámetros

**Los primeros parámetros que defina establecerán el **área** y el **intervalo de tiempo** sobre los cuales desea calcular los compuestos anuales.**

* ```studyArea``` - una colección de características, característica o geometría para especificar los límites de nuestra área de interés

* ```startJulian``` - la primera fecha juliana que se incluye en los compuestos anuales, utilizada para especificar una temporada en particular si se desea
* ```endJulian``` - la última fecha juliana para incluir en los compuestos anuales, utilizada para especificar una temporada en particular si se desea

    Esto apoya la envoltura para los trópicos y el hemisferio sur. Si está utilizando envoltura y la mayoría de los días ocurren en el segundo año, system:time_start será el 1 de junio de ese año de forma predeterminada. De lo contrario, todo system:time_starts se establecerá de forma predeterminada el 1 de junio del año determinado.

* ```startYear``` - el primer año para calcular los compuestos anuales
* ```endYear``` - el último año para calcular los compuestos anuales

Debe proporcionar al menos 3 años para que los métodos de series temporales funcionen bien. Si proporciona estadísticas precalculadas para el enmascaramiento de la nube mediante cloudScore y TDOM, esto no importa.

A continuación, utilizará un período de 3 años para explorar cómo se fabrican los composites. Después de determinar qué parámetros son apropiados, creará compuestos para todo el período de tiempo para el cual los datos Landsat están disponibles.

In [15]:
# Define user parameters:

# Specify study area
paises = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
mexico = paises.filter(ee.Filter.stringContains('ADM0_NAME','Mexico'))
studyArea = paises.filter(ee.Filter.stringContains('ADM1_NAME','Tabasco'))

# ¿Cuándo es el mejor rango de fechas para México? 
# Actualice el startJulian y endJulian
startJulian = 1
endJulian = 365

# Especifique los años de inicio y finalización de todos los análisis
startYear = 2009
endYear = 2011

# De forma predeterminada, no ejecute el método de enmascaramiento de sombras de nubes Temporal Dark Outlier Mask (TDOM)
# volveremos a TDOM más tarde
applyTDOMLandsat = False
applyTDOMSentinel2 = False
print('Hecho')

Done


#### Producir Compuestos
Usando los parámetros que especificó anteriormente, creará una serie de compuestos.

Puede revisar todos los parámetros y sus valores predeterminados en el [getCombinedLandsatSentinel2Wrapper](https://github.com/gee-community/geeViz/blob/master/examples/getCombinedLandsatSentinel2Wrapper.py).


In [16]:
#Llame a la función de contenedor maestro para obtener escenas y compuestos de Landsat
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2
                                                          )

#Separar en escenas y composiciones para su posterior análisis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Get Processed Landsat: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Only including SLC On Landsat 7
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Running Chastain et al 2019 harmonization


Tenga en cuenta que la configuración predeterminada aplica el [Fmask Cloud Mask](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L1111) y [Fmask Shadow Mask](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L1133) 
para Landsat y aplicar [Cloud Probability](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/getImagesLib.py#L668) para imágenes Sentinel-2.

Siga los hipervínculos anteriores para ver cómo se implementan estas funciones en geeViz.

#### Añadir al mapa
El `geeView` módulo de la `geeViz` paquete proporciona funciones para crear un visor de mapas interactivo fuera del editor de código y Playground de Google Earth Engine. Generalmente, `Map` funciones en `geeView` siga la misma sintaxis que los comandos de javascript Earth Engine en el patio de juegos.

Para obtener más información y ejemplos de capacidades de geeView, visite el [geeView example](https://github.com/gee-community/geeViz/blob/master/examples/geeViewExample.py) en geeViz github.

El visor de mapas interactivo aparecerá debajo.

In [17]:
Map.clearMap()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'Default Params {}-{}'.format(startJulian,endJulian),'True')
Map.addLayer(studyArea,{},'Study Area')
Map.centerObject(studyArea,10)
Map.turnOnInspector()
Map.view()

Adding layer: Default Params 1-365
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://twrlgjdgrwm-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


#### Inspeccionar las salidas

Mira el mapa que apareció arriba. Es posible que deba hacer clic en el botón al lado de la capa titulada "Parámetros predeterminados" para poder ver la capa. Haga clic en el botón de reproducción
![play button](https://github.com/redcastle-resources/lcms-training/blob/main/img/play-button.png?raw=1) para mostrar un lapso de tiempo de todos los compuestos anuales creados.  

**Está claro que los parámetros predeterminados no funcionan muy bien en esta área.**
Pocos, si es que alguno, de los años mostrados tienen imágenes completas para el compuesto anual. Faltan datos para grandes franjas de la imagen.

Aunque el panel de resultados de arriba indica las operaciones que se aplicaron para crear los compuestos, no le brinda la lista completa de parámetros utilizados para crear el compuesto. Puede acceder a los parámetros que se utilizaron a través de las propiedades de la colección devuelta.

Ejecute el bloque de código siguiente para imprimir los parámetros utilizados y mostrarlos en una tabla. Descomente la segunda línea si desea ver cómo se ve el atributo de propiedades sin procesar.

In [18]:
#Parámetros de impresión utilizados
#Los parámetros se almacenan en las propiedades de la colección devuelta
props = processedScenes.toDictionary().getInfo()
display(pd.DataFrame(list(props.values()),index = list(props.keys()),columns = ['Value']))

print('Hecho')

,Value
applyCloudProbability,True
applyCloudScoreLandsat,False
applyCloudScorePlus,False
applyCloudScoreSentinel2,False
applyFmaskCloudMask,True
applyFmaskCloudShadowMask,True
applyFmaskSnowMask,False
applyQABand,False
applyShadowShift,False
applyTDOMLandsat,False


Done


### 2.2.2: Incluye Landsat 7
Puesto que no hay muchas imágenes disponibles en esta área durante estos años, intentemos agregar Landsat 7. Nuestro valor predeterminado es excluir Landsat 7 de las composiciones de imágenes debido a la [Landsat 7 scan line correction failure](https://www.usgs.gov/landsat-missions/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con).

#### Agregar un nuevo parámetro

Para incluir Landsat 7, agregará un nuevo parámetro a la función contenedora.

``` includeSLCoffL7 ``` - por defecto: **False**

Ejecute el bloque de código siguiente para agregar el nuevo parámetro.

In [19]:
#agregar el parámetro LS 7
includeSLCOffL7 = True
print('Hecho')

Done


#### Llamar nuevamente a la función contenedora, incluido el nuevo parámetro

Ahora, vuelva a ejecutar la misma función contenedora. Tenga en cuenta que todos los parámetros son iguales, con la adición de su nuevo `includeSLCOffL7` parámetro.

In [20]:
#Llame a la función de contenedor maestro para obtener escenas y compuestos de Landsat
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2)


#Separar en escenas y composiciones para su posterior análisis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Get Processed Landsat: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Including All Landsat 7
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Running Chastain et al 2019 harmonization


#### Añadir al mapa

Ejecute el bloque de código a continuación para agregar los compuestos que creó al mapa nuevamente para inspeccionar sus resultados.

In [21]:
#Desactivar capas de la iteración anterior
Map.turnOffAllLayers()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 added {}-{}'.format(startJulian,endJulian),True)


# Evitar que el mapa se centre en el área de estudio
Map.mapCommandList = []

# Enciende el inspector
Map.turnOnInspector()

Map.view()

Adding layer: L7 added 1-365
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://twrlgjdgrwm-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


#### Inspeccionar las salidas
Active las capas en el visor de mapas si es necesario.

Alternar entre los parámetros predeterminados y los nuevos compuestos con Landsat 7 agregado. Acércate para ver los compuestos con más detalle.

Notarás que esto ayuda a llenar los vacíos, pero introduce muchos artefactos relacionados con la nube.

## 2.3: Mejorar el enmascaramiento de la nube
A continuación mejorarás el enmascaramiento de la nube. [Fmask](https://www.sciencedirect.com/science/article/abs/pii/S0034425714005069) se utiliza de forma predeterminada para Landsat, pero este algoritmo omite algunas nubes.

Probaremos la adición en el [cloudScore](https://developers.google.com/earth-engine/guides/landsat#simple-cloud-score) método para enmascarar nubes en Landsat.

Lo hacemos estableciendo el ```applyCloudScoreLandsat``` parámetro

### 2.3.1: Agregar parámetro de enmascaramiento de nube

Ejecute el bloque de código siguiente para agregar el `applyCloudScoreLandsat` parámetro.

CloudScore lucha con el error de comisión sobre superficies brillantes (enmascarando áreas frías extremadamente brillantes como nubes). El `performCloudScoreOffset` El método puede reducir este error al encontrar si un píxel generalmente tiene un cloudScore alto y ajustar el umbral de cloudScore para esos píxeles. Este método funciona bien en áreas con muchas observaciones y potencial para superficies frías y brillantes (desiertos fríos, áreas nevadas, etc.), pero falla en áreas con datos limitados y/o áreas que en realidad siempre están nubladas. Por lo tanto, querremos configurar esto en `False` sobre Tabasco México

In [22]:
# Establecer cloudScore
applyCloudScoreLandsat = True

# Establecer performCloudScoreOffset
performCloudScoreOffset = False

print('Hecho')

Done


### Ejecutar la función contenedora principal

Manteniendo todos los demás parámetros iguales, ejecute el `getLandsatAndSentinel2HybridWrapper` funcionar nuevamente. Esta vez, agregue los nuevos parámetros.

In [23]:
#Llame a la función de contenedor maestro para obtener escenas y compuestos de Landsat
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2,
                                                           applyCloudScoreLandsat=applyCloudScoreLandsat,
                                                            performCloudScoreOffset = performCloudScoreOffset)


#Separar en escenas y composiciones para su posterior análisis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Get Processed Landsat: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Including All Landsat 7
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2009 , End date: Dec 31 2011
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Running Chastain et al 2019 harmonization


### Añadir al mapa

Ejecute el bloque de código a continuación para agregar las nuevas capas al mapa.

In [ ]:
#Desactivar capas de la iteración anterior
Map.turnOffAllLayers()


Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 and CloudScore added {}-{}'.format(startJulian,endJulian),True)


Map.view()

Adding layer: L7 and CloudScore added 1-365
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://bm9sk1jqt3-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


### Inspeccionar
Activa las capas en el visor de arriba. Inspeccione la salida de **L7 y cloudScore**. Observe que estas capas se ven mucho mejor y tienen menos artefactos de enmascaramiento de nubes que las capas anteriores.

Sin embargo, todavía puedes ver algunas áreas oscuras. Es probable que se deban a sombras de nubes que no fueron enmascaradas: la máscara de sombra de nubes de Fmask normalmente no enmascara todas las sombras de nubes.

Para enmascarar la sombra de las nubes, utilizamos un método llamado [Temporal Dark Outlier Mask (TDOM)](https://www.mdpi.com/2072-4292/10/8/1184) que funciona bien para enmascarar las sombras de las nubes.



### 2.3.2: Utilice la máscara de valor atípico oscuro temporal (TDOM) para enmascarar las sombras de las nubes
El método Temporal Dark Outlier Mask (TDOM) para enmascarar las sombras de las nubes calcula la media y la desviación estándar de las bandas NIR y SWIR sobre un píxel en una serie de tiempo. El algoritmo encuentra píxeles oscuros ((SWIR1 + NIR) < 0.35) que son valores atípicos (< -1 devitación estándar) para determinar qué píxeles es probable que sean sombras de nubes (píxeles oscuros que normalmente no son oscuros).

Encuentran [details of the TDOM method](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L1141) en la biblioteca de geeViz.

#### Establecer el parámetro TDOM

 - ``` applyTDOM ``` - por defecto = **False**

Ejecute el bloque de código siguiente para configurar el parámetro TDOM.

In [24]:
#Establecer TDOM en verdadero
applyTDOMLandsat = True
applyTDOMSentinel2 = True
print('Hecho')

Done


#### Ejecutar la función de contenedor maestro

Llame a la función de contenedor maestro para obtener escenas y compuestos de Landsat
Para identificar valores atípicos oscuros, ampliaremos las fechas 3 años en cualquier dirección para obtener una muestra más grande

In [25]:
#Llame a la función de contenedor maestro para obtener escenas y compuestos de Landsat
#Para identificar valores atípicos oscuros, ampliaremos las fechas 3 años en cualquier dirección para obtener una muestra más grande
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear - 3,
                                                           endYear + 3,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyCloudScoreLandsat=applyCloudScoreLandsat,
                                                           applyTDOMLandsat = applyTDOMLandsat,
                                                           applyTDOMSentinel2 = applyTDOMSentinel2)


#Separar en escenas y composiciones para su posterior análisis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jan 01 2006 , End date: Dec 31 2014
Get Processed Landsat: 
Start date: Jan 01 2006 , End date: Dec 31 2014
Including All Landsat 7
Applying Cloud Score
Computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jan 01 2006 , End date: Dec 31 2014
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Applying Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Running Chastain et al 2019 harmonization


### Añadir al mapa

Ejecute el bloque de código a continuación para agregar las nuevas capas al mapa. Esta capa puede tardar más en ejecutarse y cargarse.

In [ ]:
#Desactivar capas de la iteración anterior
Map.turnOffAllLayers()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'CloudScore and TDOM added {}-{}'.format(startJulian,endJulian),True)

Map.view()

Adding layer: CloudScore and TDOM added 1-365
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd /content
Colab Proxy URL: https://bm9sk1jqt3-496ff2e9c6d22116-1232-colab.googleusercontent.com/geeView/?accessToken=None


#### Inspeccionar
Activa la nueva capa. Notarás que esto limpia mucho la máscara de la nube.

#### TDOM es computacionalmente intensivo
Sin embargo, si bien TDOM es un excelente método de enmascaramiento de sombras de nubes, requiere un poco de computación, ya que calcula la media y la desviación estándar de una gran pila de datos.

Para evitar volver a calcular las estadísticas, almacenamos estadísticas calculadas previamente. Para el proyecto LCMS, hemos calculado previamente estadísticas para CONUS, AK, HI, PuertoRico y USVI. Usaremos estas estadísticas precalculadas.


Es opcional que utilice estadísticas precalculadas. Si no se proporciona ninguno, las estadísticas de TDOM se calcularán sobre la marcha.

Puede crear y almacenar estadísticas TDOM precalculadas aplicando únicamente el enmascaramiento de nubes durante un período de tiempo y calculando la media y la desviación estándar de las bandas NIR y SWIR1.


#### Inspeccionar estadísticas TDOM sin procesar y precalculadas

Ejecute el siguiente código para cargar las estadísticas de TDOM precalculadas y examinarlas en el mapa.

In [ ]:
# clear map
Map.clearMap()

# cargar estadísticas tdom precalculadas
landsat_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Landsat_TDOM_Stats_1984_2021')\
                    .select(['Landsat_nir_.*','Landsat_swir1_.*'])\
                    .divide(10000)
s2_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Sentinel2_TDOM_Stats_2015_2021')\
                    .select(['Sentinel2_nir_.*','Sentinel2_swir1_.*'])\
                    .divide(10000)
Map.addLayer(landsat_tdom_stats,{'min':0.15,'max':0.35,'bands':'Landsat_nir_mean','palette':'222,080'},'Landsat TDOM Stats')
Map.addLayer(s2_tdom_stats,{'min':0.15,'max':0.35,'bands':'Sentinel2_nir_mean','palette':'222,080'},'Sentinel 2 TDOM Stats')

Map.turnOnInspector()
Map.view()


Adding layer: Landsat TDOM Stats
Adding layer: Sentinel 2 TDOM Stats
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


## 2.4: Preparar compuestos finales y exportar
Ahora que ha repasado algunos de los parámetros que se pueden cambiar para los compuestos, usaremos Landsat y Sentinel2 para obtener los mejores compuestos posibles.

Aquí, configurará manualmente todos los parámetros para el `getLandatAndSentinel2HybridWrapper`.

### Proyecciones

A algunas proyecciones comunes se accede a través de `getImagesLib.common_projection`
LCMS utiliza la misma cuadrícula de ajuste (transformación) y proyección (crs) que el USGS NLCD. A menudo se le llama USGS Albers. Puede cambiar esto a cualquier proyección que desee según su área de estudio usando códigos EPSG o formato WKT.


### Establecer parámetros y exportar

Revisa el [documentation](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L3280) para recuperar los valores predeterminados para cada uno de estos parámetros.

Tenga en cuenta en la función a continuación dónde los parámetros son diferentes de los predeterminados.

Ejecute el siguiente bloque de código para preparar y exportar compuestos.

In [27]:
# Ejecute obtener imágenes, establezca parámetros manualmente y exporte
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(\

    # Área y tiempo de estudio
    studyArea = studyArea,
    startYear = 1984,
    endYear = 2022,
    startJulian = 152,
    endJulian = 151,

    # Opciones de ventana móvil
    timebuffer =  0,# Si se necesita una ventana móvil de varios años, utilice un búfer de tiempo > 0 (por ejemplo, un búfer de tiempo = 1 dará como resultado una ventana móvil de 3 años)
    weights =  [1],# Para complementar el margen de tiempo, establezca ponderaciones para cada año de la ventana móvil. Esto permite que el año central tenga más peso (por ejemplo, [1,3,1] ponderaría el año central si el buffer de tiempo = 1 3 veces más que los años buffer)

    # Métodos compuestos e imágenes de entrada
    compositingMethod = 'medoid', # Especifique mediana o medioide. Medoid permite almacenar solo las bandas espectrales y auxiliares sin índices
    toaOrSR = 'TOA', # Se recomienda utilizar TOA únicamente al combinar Landsat y Sentinel-2, ya que los datos de Sentinel-2 SR en GEE están corregidos según el terreno y Landsat no.
    includeSLCOffL7 = True, # Si se debe incluir Landsat 7 después de mayo de 2003. Se pueden encontrar artefactos en la línea de exploración en compuestos cuando es Verdadero
    defringeL5 = True, # Si es necesario deshacerse de los bordes marginales que se encuentran principalmente en Landsat 4 y 5
    landsatCollectionVersion = 'C2', # Para usar la Colección 1 o 2. Solo use C1 si realiza un trabajo heredado antes de 2022

    # Enmascaramiento de nubes y sombras
    # Habrá un parámetro para aplicar CloudScore+ a Sentinel-2 una vez que esté disponible para todos los datos de S2
    applyQABand = False, # Si se debe aplicar la máscara de control de calidad Sentinel-2 predeterminada: este método es terrible y no debe usarse
    applyCloudProbability = True,# Si se debe aplicar S2Cloudless a los datos de Sentinel-2. Este método funciona muy bien
    applyShadowShift = False, # Este método está en gran medida obsoleto
    applyCloudScoreLandsat = True, # Usamos esto ya que fMask tiene dificultades para enmascarar las nubes
    applyCloudScoreSentinel2 = False, # Normalmente no utilizamos cloudScore para Sentinel-2 ya que S2Cloudless funciona muy bien.
    applyTDOMLandsat = True, # Usamos TDOM para Landsat ya que el método de enmascaramiento de sombras de nubes fMask tiene muchos problemas
    applyTDOMSentinel2 = True, # Este es el único método de enmascaramiento de sombras de nubes Sentinel-2 fácilmente disponible (a partir de agosto de 2023)
    applyFmaskCloudMask = True,
    applyFmaskCloudShadowMask = True,
    applyFmaskSnowMask = False,
    cloudHeights = ee.List.sequence(500,10000,500), # Esto complementa a applyShadowShift y está en gran medida obsoleto,
    cloudScoreThresh = 10,# El umbral aplicado al cloudScore para Landsat y/o Sentinel-2
    performCloudScoreOffset = False,# Establezca esto en Falso en áreas con datos limitados y/o superficies frías brillantes limitadas
    cloudScorePctl = 10, # Si performCloudScoreOffset está configurado en True, este es el percentil de los cloudScores utilizados para ver si un área normalmente tiene un cloudScore alto.
    zScoreThresh = -1, # El umbral de puntuación TDOM Z para encontrar valores atípicos oscuros
    shadowSumThresh = 0.35,# La suma de ShadowSumBands debe estar por debajo de esto para que se considere oscura
    contractPixels = 1.5,# Radio de píxel del búfer interno para eliminar la sal y la pimienta
    dilatePixels = 3.5, # Radio de píxel del búfer exterior para dilatar la máscara de sombra de la nube
    shadowSumBands = ['nir','swir1'], # Bandas a utilizar para que TDOM encuentre valores atípicos oscuros
    cloudProbThresh = 40, # El umbral de probabilidad de nube de S2Cloudless

    # Procesamiento de imágenes
    landsatResampleMethod = 'bicubic',
    sentinel2ResampleMethod = 'bicubic',
    convertToDailyMosaics = True, # Importante si el recuento de observaciones compuestas es importante ya que Sentinel-2 se superpone con sus salidas en mosaico
    runChastainHarmonization = False, # Establezca en True para realizar la armonización de regresión entre Landsat y Sentinel 2
    correctIllumination = False, # Método para iluminar sombras de colinas. Esto está en gran medida en desuso y no se utiliza
    correctScale = 250,

    # Parámetros de exportación
    exportComposites = True,# Si se exportan los compuestos resultantes
    outputName = 'Landsat-Sentinel2-Hybrid',
    exportPathRoot = export_composite_collection,
    crs = getImagesLib.common_projections['NLCD_CONUS']['crs'],
    transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
    scale = None,
    overwrite = False,# Ya sea para sobrescribir compuestos existentes

    # TDOM y cloudscore precalculado y estadísticas 
    preComputedLandsatCloudScoreOffset = None,
    preComputedLandsatTDOMIRMean = None,
    preComputedLandsatTDOMIRStdDev = None,
    preComputedSentinel2CloudScoreOffset = None,
    preComputedSentinel2TDOMIRMean = None,
    preComputedSentinel2TDOMIRStdDev = None
                                                           )


print('Hecho')

Get Processed Landsat and Sentinel2 Scenes: 
Start date: May 31 1984 , End date: May 31 2023
Get Processed Landsat: 
Start date: May 31 1984 , End date: May 31 2023
Defringing L4 and L5
Including All Landsat 7
Setting resample method to  bicubic
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: May 31 1984 , End date: May 31 2023
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting resample method to  bicubic
Converting S2 data to daily mosaics
Applying Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1986_1986_152_151
Exporti

### Gestionar tareas de exportación
Estas tareas de exportación pueden tardar desde unas pocas horas hasta unos días en ejecutarse, según el tamaño de su área de estudio. Puede resultar útil gestionar y observar las tareas después de haberlas configurado en ejecución.

A continuación se muestran algunos fragmentos de código que se pueden usar para administrar e inspeccionar las tareas de exportación.

Enlace a la documentación sobre estos fragmentos de código.

In [ ]:
# Si desea realizar un seguimiento de las tareas, utilice esto:
tml.trackTasks2()

# Advertencia!!! - Utilice estos métodos solo si es necesario; pueden detener y eliminar el trabajo
# Si desea cancelar todas las tareas en ejecución, puede utilizar esta función
# tml.batchCancel()

# Si desea vaciar la colección de todas las imágenes
# aml.batchDelete(export_composite_collection, type = 'imageCollection')

print('Hecho')

37 tasks ready 2023-11-08 22:57:42
2 tasks running 2023-11-08 22:57:42
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:24']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:24']


37 tasks ready 2023-11-08 22:57:47
2 tasks running 2023-11-08 22:57:47
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:30']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:30']


37 tasks ready 2023-11-08 22:57:53
2 tasks running 2023-11-08 22:57:53
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:36']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:36']


37 tasks ready 2023-11-08 22:57:58
2 tasks running 2023-11-08 22:57:58
Running names:
['Landsat-Sentinel2-Hybrid_TOA_medoid_1985_1985_152_151', '0:04:41']
['Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151', '0:04:41']




### View composites as they are completed
After your composites have been exported, you can view the composites in your folder.

In [ ]:
# Ver compuestos a medida que se completan
composites = ee.ImageCollection(export_composite_collection)

# Si sus compuestos aún no se han exportado, puede utilizar la siguiente línea de código para inspeccionar compuestos precalculados
composites = ee.ImageCollection(f'{pre_baked_path_root}/lcms-training_module-2_composites')

print('Hecho')

Done


### Inspeccionar los resultados

Explore los compuestos y algunos atributos que describen la calidad de los compuestos.

Observe cómo el número de observaciones aumenta y el sensor predominante cambia con el tiempo

In [ ]:
# clear the map
Map.clearMap()

# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.addTimeLapse(composites, getImagesLib.vizParamsFalse10k,'Composites')

# By looking at the Sensor that is used, you can see
# with the introduction of Landsat 8 in 2013 and then Sentinel 2 in 2016-2017,
# composites become much better quality
Map.addTimeLapse(composites.select('sensor'),{'min':4,'max':22,'palette':'088,808','classLegendDict':{'Landsat (Landsat 5 = 5, Landsat 7 = 7, etc...)':'088','Sentinel2 (Sentinel 2A = 21, Sentinel 2B = 22, etc....)':'808'},'queryDict':{4:'L4',5:'L5',7:'L7',8:'L8',9:'L9',21:'S2a',22:'S2b'}},'Sensor')
Map.addTimeLapse(composites.select('compositeObsCount'),{'min':3,'max':10,'palette':'D0D,0D0'},'Composite Observation Counts')
Map.turnOnInspector()
Map.view()
# Now that we have exported composites, we will use them in the LandTrendr temporal segmentation algorithm

Adding layer: Composites
Adding layer: Sensor
Adding layer: Composite Observation Counts
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1232/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


## Desafío del laboratorio 2:

**Para los usuarios de Qwiklabs**, se evaluará su finalización en la parte "Verificar mi progreso" al final de la práctica 2.

1. Cree una serie temporal compuesta de medoid sin nube de 2020 a 2023 para los días 1 a 150 utilizando los métodos que utilizamos en este módulo.

    * Utilice lo siguiente como su área de estudio:

    ```python
          ee.Geometry.Polygon(
                [[[-65.91075380517458, 18.265940533464065],
                  [-65.91075380517458, 18.236922027462644],
                  [-65.87350328637575, 18.236922027462644],
                  [-65.87350328637575, 18.265940533464065]]], None, False)
      ```
      <br>
   
   * Selecciona las siguientes bandas:`["green","red","nir","swir1","swir2","NBR"]`

<br>

2. Extraiga los valores de los tres compuestos anuales para una ubicación de punto determinada.

    * Utilice un punto con estas coordenadas: `([-65.88860948754763, 18.249964652264815])`
            
    * Utilice la siguiente función: `g2p.extractPointValuesToDataFrame`
    <br><br>
    Ejemplo:
    ```python
          extracted_values = g2p.extractPointValuesToDataFrame(
          composite,
          ee.Geometry.Point([-65.88860948754763, 18.249964652264815]),
          scale=30,
          crs = "EPSG:5070",
          transform = None,
          reducer = ee.Reducer.first(),
          includeNonSystemProperties = False,
          includeSystemProperties=True
          )
    ```
<br>    

3. Guarde los valores extraídos en un archivo csv.

   * Guarde csv en esta ruta: `"/tmp/challenge/module_2_challenge_answer.csv"`
     * **Nota: La ruta al csv debe coincidir exactamente con la ruta anterior.**
    <br>
    
    * Creer `"/tmp/challenge"` carpeta si ya no existe.
      
        Ejemplo:
    ```python
        out_csv = "/tmp/challenge/module_2_challenge_answer.csv"
        if not os.path.exists(os.path.dirname(out_csv)):os.makedirs(os.path.dirname(out_csv))
    ```
<br>

4.  Checar que el csv de salida exista.
    
    * Ejemplo:
    ```python
        print(os.path.exists(out_csv))
    ```
<br>

In [ ]:
# Pon tu código de desafío aquí


## Hecho con el laboratorio 2

Ahora que hemos exportado compuestos, los usaremos en el algoritmo de segmentación temporal LandTrendr en el módulo 3.